# Quickstart
to get features such as:
- hybrid search
- multivector search
- filtered search
- etc out of the box

# What I Need

- Project & API Key (grab your API key from https://cloud.relevance.ai/ in the settings area)
- Python 3
- Relevance AI Installed as shown below. You can also visit our [Installation guide](https://docs.relevance.ai/docs)

# Installation Requirements

In [ ]:
# RelevanceAI installation
!pip install -U RelevanceAI[notebook]==0.33.2

# Vectorhub installation for quick access to Sentence Transformers
!pip install vectorhub[encoders-text-tfhub]

# Set up

To use Relevance AI, a client object must be instantiated. This needs an API_key and a project name. These can be generated/access directly at https://cloud.relevance.ai/ or simply by running the cell below and following the link and the guide:


In [ ]:
from relevanceai import Client 

"""
Running this cell will provide you with 
the link to sign up/login page where you can find your credentials.
Once you have signed up, click on the value under `Authorization token` 
in the API tab and paste it in the appreared Auth token box below
"""

client = Client()

# Vector search

## 1) Data

For this quickstart we will be using a sample e-commerce dataset. Alternatively, you can use your own dataset for the different steps.

In [ ]:
import pandas as pd
from relevanceai.datasets import get_ecommerce_dataset_clean

# Retrieve our sample dataset. - This comes in the form of a list of documents.
documents = get_ecommerce_dataset_clean()

pd.DataFrame.from_dict(documents).head()

In [ ]:
documents[0].keys()

## 2) Encode

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

# Here we load the model and define how we encode
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')

# First we define how we encode the queries
def encode_query(query: str):
    return module.signatures['question_encoder'](tf.constant([query]))['outputs'][0].numpy().tolist()

# We then want to define how we encode the answers
def encode_answer(answer: str):
    return module.signatures['response_encoder'](
        input=tf.constant([answer]), 
        context=tf.constant([answer]))['outputs'][0].numpy().tolist()


In [ ]:
from tqdm.auto import tqdm

for d in tqdm(documents):
    d['product_title_use_qa_vector_'] = encode_answer(d['product_title'])

## 3) Insert

Uploading our documents into a dataset called `quickstart_tfhub_qa`.
Note that each document should have a field called '_id'. Here, we generate a unique identifier using the Python uuid package.

In [ ]:
import uuid

for d in documents:
  d['_id'] = uuid.uuid4().__str__()    # Each document must have a field '_id'

# Insert the documents into a dataset called quickstart_tfhub_qa.
dataset_id = "quickstart_tfhub_qa"
df = client.Dataset(dataset_id)
df.delete()
df.insert_documents(documents)

## 4) Search

Note that our dataset includes vectors generated by Universal Sentence Encoder. Therefore, in this step, we first vectorize the query using the same encoder to be able to search among the similarly generated vectors.

In [ ]:
query = "for my baby daughter"
query_vector = encode_query(query)

In [ ]:
results = df.vector_search(
    multivector_query=[
        {
            "vector": query_vector,
            "fields": ["product_title_use_qa_vector_"]}
    ],
    page_size=5
)

You can now visit the dashboard at https://cloud.relevance.ai/sdk/search


In [ ]:
from relevanceai import show_json
show_json(
    results['results'],
    text_fields=["product_title"],
    image_fields = ["product_image"]
)

,product_image,product_title,_id
0,,Cienta Girl (Toddler) '108048' Basic Textile Boots,97dae01b-3fdc-4555-8540-ca429098d94f
1,,Nina Kids Girl (Toddler) 'Nesa' Man-Made Sandals,7e29fab3-bcd9-456f-aa26-adac15ea3cd4
2,,Evenflo Dottie Rose Convertible 3-in-1 High Chair,40d1a884-a2b7-4cae-8e91-cb2c8350bc9f
3,,Layla Chenille Bedspread (Shams Sold Separately),6b6e0ee6-a465-44f2-a6b3-6a545bfdc371
4,,Osh Kosh Girl (Toddler) 'Rapid-14' Man-Made Sandals (Size 7 ),147f7fae-e4e6-4f81-a7f2-d07f311c9ca0
